In [1]:
import pandas as pd
import json
import os

In [2]:
def get_character_index(story_lines):
  indexes = []
  cur_index = 0
  for i in range(len(story_lines)):
    indexes.append(cur_index)
    cur_index += len(story_lines[i])
  indexes.append(cur_index)
  return indexes

In [3]:
schoolmistress_story_paragraphs = open('schoolmistress_paragraphs.txt', 'r').readlines()
schoolmistress_story_paragraphs_indexes = get_character_index(schoolmistress_story_paragraphs)
schoolmistress_story_sentences = open('schoolmistress.txt', 'r').readlines()
schoolmistress_story_sentences_indexes = get_character_index(schoolmistress_story_sentences)

el_story_paragraphs = open('expensivelessons_paragraphs.txt', 'r').readlines()
el_story_paragraphs_indexes = get_character_index(el_story_paragraphs)
el_story_sentences = open('expensivelessons.txt', 'r').readlines()
el_story_sentences_indexes = get_character_index(el_story_sentences)

In [4]:
def update_indexes(highlight, indexes):
  for i in range(len(highlight) - 1):
    highlight[i]['start'] = highlight[i]['start'] + indexes[highlight[-1]['paragraph']]
    highlight[i]['end'] = highlight[i]['end'] + indexes[highlight[-1]['paragraph']]
  return highlight

In [5]:
def parse_highlights(filename, story_sentences, story_paragraphs, story_sentences_indexes, story_paragraphs_indexes):
  highlight_file = open(f'./highlighting_logs/{filename}.log', 'r')
  CATEGORY_MAP = {'None': 0, 'Present': 1, 'Confused': 2, 'Curious': 3, 'Connected': 4, 'Other': 5}
  lines = highlight_file.readlines()
  highlights = [None] * len(story_paragraphs)
  for line in lines:
    start = line.find('Highlight:')
    if start != -1:
      highlight = json.loads(str(line[start + len('Highlight: '):].replace("'", '"')), )
      highlight_absolute_indexes = update_indexes(highlight, story_paragraphs_indexes)
      highlights[highlight[-1]['paragraph']] = highlight_absolute_indexes
  sentence_highlights = [{'category': 0, 'proportion': 0.0}] * len(story_sentences)
  for i in range(len(highlights)):
    if highlights[i] != None:
      for k in range(len(highlights[i]) - 1):
        start_sent = None
        end_sent = None
        for p in range(len(story_sentences_indexes) - 1):
          if highlights[i][k]['start'] >= story_sentences_indexes[p] and highlights[i][k]['start'] < story_sentences_indexes[p+1]:
            start_sent = p
          if highlights[i][k]['end'] >= story_sentences_indexes[p] and highlights[i][k]['end'] < story_sentences_indexes[p+1]:
            end_sent = p
          if start_sent != None:
            end_index = story_sentences_indexes[p+1] if end_sent == None else highlights[i][k]['end']
            start_index = story_sentences_indexes[p] if story_sentences_indexes[p] > highlights[i][k]['start'] else highlights[i][k]['start']
            proportion = (end_index - start_index) / (story_sentences_indexes[p+1] - story_sentences_indexes[p])
            # print(f"start: {start_index}, end: {end_index}, proportion: {proportion}")
            if proportion > sentence_highlights[p]['proportion']:
              sentence_highlights[p] = {'category': CATEGORY_MAP[highlights[i][k]['tag']], 'proportion': proportion}
          if end_sent != None:
            break
  
  highlight_df = pd.DataFrame.from_dict(sentence_highlights)
  highlight_df.to_csv(f'./highlights/{filename}.csv')

In [6]:
HIGHLIGHT_DIR = "./highlighting_logs/"

for filename in os.listdir(HIGHLIGHT_DIR):
    f = os.path.join(HIGHLIGHT_DIR,filename)
    if os.path.isfile(f):
        filename = f.replace(HIGHLIGHT_DIR, '').replace('.log', '')
        if 'schoolmistress' in filename:
          parse_highlights(filename, schoolmistress_story_sentences, schoolmistress_story_paragraphs, schoolmistress_story_sentences_indexes, schoolmistress_story_paragraphs_indexes)
        elif 'el' in filename:
          parse_highlights(filename, el_story_sentences, el_story_paragraphs, el_story_sentences_indexes, el_story_paragraphs_indexes)

